In [171]:
import requests
import json
import boto3

In [172]:

ACCESS_KEY = ''
SECRET_KEY = ''
API_KEY = ''


In [173]:
# S3
REGION = 'ca-central-1'
BUCKET = 'quetzal-immense'

# api gateway
url = 'https://lud5uqi5j5.execute-api.ca-central-1.amazonaws.com/dev'
stateMachineArn = 'arn:aws:states:ca-central-1:142023388927:stateMachine:quetzal-immense'

In [174]:
headers = {
        'Accept': '*/*',
        'Content-Type':'application/json',
        'x-api-key' : API_KEY,
}

# start model

All parameters passed to the function or in params

In [175]:
params = {"param1":"test","param2":3}
launcher_arg = {'training_folder': '/tmp','params': params}
inputs = { 
        'scenario_path_S3': 'test/',
        'choice':'default',
        'launcher_arg': launcher_arg,
        'metadata' : {"user_email": "test@test.com"}
}
inputs = json.dumps(inputs)

In [176]:
data = { "stateMachineArn": stateMachineArn, 'input':inputs }
data = json.dumps(data)
data


'{"stateMachineArn": "arn:aws:states:ca-central-1:142023388927:stateMachine:quetzal-immense", "input": "{\\"scenario_path_S3\\": \\"test/\\", \\"choice\\": \\"default\\", \\"launcher_arg\\": {\\"training_folder\\": \\"/tmp\\", \\"params\\": {\\"param1\\": \\"test\\", \\"param2\\": 3}}, \\"metadata\\": {\\"user_email\\": \\"test@test.com\\"}}"}'

In [177]:
resp =  requests.post(url,data=data,headers=headers)
resp.ok

True

In [178]:
resp.json()

{'executionArn': 'arn:aws:states:ca-central-1:142023388927:execution:quetzal-immense:9127a24c-a68b-41d1-820a-bfc958e7bbba',
 'startDate': 1707507501.651}

In [179]:
# get executionARN for polling
resp.json()['executionArn']

'arn:aws:states:ca-central-1:142023388927:execution:quetzal-immense:9127a24c-a68b-41d1-820a-bfc958e7bbba'

# get status (polling)

In [180]:
executionArn = resp.json()['executionArn']
data = { "executionArn": executionArn }
data=json.dumps(data, ensure_ascii=False).encode('utf-8')
resp =  requests.post(url+'/describe',data=data,headers=headers)
resp.ok

True

In [181]:
resp.json()

{'executionArn': 'arn:aws:states:ca-central-1:142023388927:execution:quetzal-immense:9127a24c-a68b-41d1-820a-bfc958e7bbba',
 'input': '{"scenario_path_S3": "test/", "choice": "default", "launcher_arg": {"training_folder": "/tmp", "params": {"param1": "test", "param2": 3}}, "metadata": {"user_email": "test@test.com"}}',
 'inputDetails': {'__type': 'com.amazonaws.swf.base.model#CloudWatchEventsExecutionDataDetails',
  'included': True},
 'name': '9127a24c-a68b-41d1-820a-bfc958e7bbba',
 'redriveCount': 0,
 'redriveStatus': 'NOT_REDRIVABLE',
 'redriveStatusReason': 'Execution is RUNNING and cannot be redriven',
 'startDate': 1707507501.651,
 'stateMachineArn': 'arn:aws:states:ca-central-1:142023388927:stateMachine:quetzal-immense',
 'status': 'RUNNING',
 'traceHeader': 'Root=1-65c67f2d-4ca7f8c3003583114b0a852b;Sampled=1'}

In [182]:
resp.json()['status']

'RUNNING'

In [183]:
import time

status =  resp.json()['status']
while status == 'RUNNING':
    resp =  requests.post(url+'/describe',data=data,headers=headers)
    status = resp.json()['status']
    print(status)
    time.sleep(2)

RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
FAILED


SUCCEEDED or ERROR

# stop

In [30]:
executionArn

'arn:aws:states:ca-central-1:142023388927:execution:quetzal-immense:dfb5ebf3-53f8-46e4-903d-0a906a467ba9'

In [31]:
data = { "executionArn": executionArn }
data = json.dumps(data)
resp =  requests.post(url+'/abort',data=data,headers=headers)
resp.ok

True

In [32]:
resp.json()

{'stopDate': 1707245846.29}

In [33]:
data = { "executionArn": executionArn }
data=json.dumps(data, ensure_ascii=False).encode('utf-8')
resp =  requests.post(url+'/describe',data=data,headers=headers)
resp.ok

True

In [34]:
resp.json()

{'executionArn': 'arn:aws:states:ca-central-1:142023388927:execution:quetzal-immense:dfb5ebf3-53f8-46e4-903d-0a906a467ba9',
 'input': '{"scenario_path_S3": "test/", "launcher_arg": {"training_folder": "/tmp", "params": {"param1": "test", "param2": 3}}, "metadata": {"user_email": "test@test.com"}}',
 'inputDetails': {'__type': 'com.amazonaws.swf.base.model#CloudWatchEventsExecutionDataDetails',
  'included': True},
 'name': 'dfb5ebf3-53f8-46e4-903d-0a906a467ba9',
 'output': '{"notebook_path":"notebooks/2_model/test.ipynb","metadata":{"user_email":"test@test.com"},"scenario_path_S3":"test/","launcher_arg":{"training_folder":"/tmp","params":{"param1":"test","param2":3}}}',
 'outputDetails': {'__type': 'com.amazonaws.swf.base.model#CloudWatchEventsExecutionDataDetails',
  'included': True},
 'redriveCount': 0,
 'redriveStatus': 'NOT_REDRIVABLE',
 'redriveStatusReason': 'Execution is SUCCEEDED and cannot be redriven',
 'startDate': 1707245836.932,
 'stateMachineArn': 'arn:aws:states:ca-cent

In [35]:
resp.json()['status'] # 'ABORTED' 

'SUCCEEDED'

# S3

# S3 structure

scenario/inputs/ <br>
scenario/outputs/<br><br>

In [184]:
session = boto3.Session(
    region_name=REGION,
    aws_access_key_id=ACCESS_KEY,
    aws_secret_access_key=SECRET_KEY)

s3 = session.client('s3')

In [185]:
response = s3.list_objects_v2(Bucket=BUCKET)

In [204]:
ls=[]
for obj in response['Contents']:
    ls.append(obj["Key"])
ls

# Run

In [191]:
inputs = { 
        'choice':'orchestrator',
        'metadata' : {"user_email": "test@test.com"}
}
inputs = json.dumps(inputs)


In [192]:
data = { "stateMachineArn": stateMachineArn, 'input':inputs }
data = json.dumps(data)
data

'{"stateMachineArn": "arn:aws:states:ca-central-1:142023388927:stateMachine:quetzal-immense", "input": "{\\"choice\\": \\"orchestrator\\", \\"metadata\\": {\\"user_email\\": \\"test@test.com\\"}}"}'

In [193]:
resp =  requests.post(url,data=data,headers=headers)
resp.ok

True

In [199]:
executionArn = resp.json()['executionArn']
data = { "executionArn": executionArn }
data=json.dumps(data, ensure_ascii=False).encode('utf-8')
resp =  requests.post(url+'/describe',data=data,headers=headers)
resp.ok

status =  'RUNNING'
while status == 'RUNNING':
    resp =  requests.post(url+'/describe',data=data,headers=headers)
    status = resp.json()['status']
    print(status)
    time.sleep(10)

RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
SUCCEEDED


In [203]:
response = s3.list_objects_v2(Bucket=BUCKET,Prefix='orchestrator/outputs')
ls=[]
for obj in response['Contents']:
    ls.append(obj["Key"])
ls

['orchestrator/outputs/volume_am.csv',
 'orchestrator/outputs/volume_pm.csv',
 'orchestrator/outputs/volume_test.csv']